# 05 - Advanced Configuration

This notebook covers advanced configuration options for customizing your experiments.

## What you'll learn:
- Complete configuration reference
- Custom model architectures
- Loss function combinations
- Advanced augmentation pipelines
- Multi-class and regression tasks

In [ ]:
import altair as alt
from altair.core.config import Config

## Configuration Structure

A complete Altair configuration has these sections:

In [ ]:
complete_config = """
# ============================================
# EXPERIMENT SETTINGS
# ============================================
experiment:
  name: "my_experiment"        # Experiment name
  project: "segmentation"      # Project grouping
  seed: 42                     # Random seed for reproducibility
  output_dir: "experiments"    # Where to save outputs

# ============================================
# MODEL CONFIGURATION
# ============================================
model:
  architecture: "unet"         # Architecture: unet, unetplusplus
  encoder: "resnet34"          # Encoder backbone (any timm model)
  encoder_weights: "imagenet"  # Pretrained weights (or null)
  num_classes: 2               # Number of output classes
  task: "binary"               # Task: binary, multiclass, regression
  in_channels: 3               # Input channels (3 for RGB)
  decoder_channels: [256, 128, 64, 32, 16]  # Decoder channel sizes
  decoder_attention: "scse"    # Attention: null, scse

# ============================================
# DATA CONFIGURATION
# ============================================
data:
  format: "png"                # Format: png, coco
  train_images: "data/train/images"
  train_masks: "data/train/masks"
  val_images: "data/val/images"
  val_masks: "data/val/masks"
  batch_size: 8
  num_workers: 4
  pin_memory: true

# ============================================
# TRAINING CONFIGURATION
# ============================================
training:
  epochs: 100
  lr: 0.001
  optimizer: "adamw"           # Optimizer: adam, adamw, sgd
  weight_decay: 0.01
  scheduler: "cosine"          # Scheduler: cosine, step, plateau
  loss: "dice"                 # Loss: bce, ce, dice, focal, combined
  gradient_clip: 1.0           # Gradient clipping (null to disable)
  amp: true                    # Automatic mixed precision

# ============================================
# AUGMENTATIONS
# ============================================
augmentations:
  train:
    - name: resize
      height: 256
      width: 256
    - name: horizontal_flip
      p: 0.5
    - name: normalize
  val:
    - name: resize
      height: 256
      width: 256
    - name: normalize

# ============================================
# CHECKPOINTING
# ============================================
checkpointing:
  monitor: "val/Dice"          # Metric to monitor
  mode: "max"                  # max or min
  save_top_k: 3                # Keep top K checkpoints
  save_last: true              # Always save last checkpoint

# ============================================
# EVALUATION
# ============================================
evaluation:
  thresh: 0.5                  # Threshold for binary prediction
  iou_high: 0.5                # Soft PQ upper bound
  iou_low: 0.05                # Soft PQ lower bound

# ============================================
# TRACKING (MLflow)
# ============================================
tracking:
  enabled: true
  uri: "mlruns"                # MLflow tracking URI
"""
print(complete_config)

## Model Architectures

### UNet with Different Encoders

In [ ]:
encoders = """
# ResNet family
encoder: "resnet18"     # Lightest, fastest
encoder: "resnet34"     # Good balance
encoder: "resnet50"     # More capacity
encoder: "resnet101"    # Even more capacity

# EfficientNet family
encoder: "efficientnet_b0"  # Smallest
encoder: "efficientnet_b3"  # Medium
encoder: "efficientnet_b5"  # Large
encoder: "efficientnet_b7"  # Largest

# Other popular encoders
encoder: "mobilenetv3_large_100"  # Mobile-friendly
encoder: "convnext_tiny"          # Modern ConvNet
encoder: "swin_tiny_patch4_window7_224"  # Vision Transformer

# Any timm model works!
# See: https://huggingface.co/timm
"""
print(encoders)

### UNet++ (Dense Skip Connections)

In [ ]:
unetpp_config = """
model:
  architecture: "unetplusplus"
  encoder: "resnet34"
  encoder_weights: "imagenet"
  num_classes: 5
  task: "multiclass"
  
  # UNet++ has nested skip connections
  # Generally better for small objects
"""
print(unetpp_config)

### Decoder Attention (SCSE)

In [ ]:
attention_config = """
model:
  architecture: "unet"
  encoder: "resnet50"
  encoder_weights: "imagenet"
  num_classes: 2
  task: "binary"
  
  # SCSE: Squeeze-and-Excitation with spatial attention
  # Improves feature recalibration
  decoder_attention: "scse"
  
  # Or disable attention
  # decoder_attention: null
"""
print(attention_config)

## Loss Functions

In [ ]:
loss_examples = """
# Binary segmentation
training:
  loss: "bce"          # Binary Cross-Entropy
  loss: "dice"         # Dice Loss (good for imbalanced)
  loss: "focal"        # Focal Loss (hard examples)
  
# Multi-class segmentation
training:
  loss: "ce"           # Cross-Entropy
  loss: "dice"         # Multi-class Dice
  loss: "focal"        # Multi-class Focal

# Regression
training:
  loss: "mse"          # Mean Squared Error
  loss: "l1"           # Mean Absolute Error

# Combined losses
training:
  loss: "combined"
  loss_weights:
    bce: 0.5
    dice: 0.5
    
# Or with different weights
training:
  loss: "combined"
  loss_weights:
    ce: 1.0
    dice: 0.5
    focal: 0.25
"""
print(loss_examples)

## Optimizers and Schedulers

In [ ]:
optimizer_examples = """
# AdamW (recommended)
training:
  optimizer: "adamw"
  lr: 0.001
  weight_decay: 0.01

# Adam
training:
  optimizer: "adam"
  lr: 0.001
  weight_decay: 0.0

# SGD with momentum
training:
  optimizer: "sgd"
  lr: 0.01
  momentum: 0.9
  weight_decay: 0.0001

# Schedulers
training:
  scheduler: "cosine"    # Cosine annealing (recommended)
  scheduler: "step"      # Step decay
  scheduler: "plateau"   # Reduce on plateau
  scheduler: null        # No scheduler
"""
print(optimizer_examples)

## Augmentation Pipelines

In [ ]:
aug_light = """
# Light augmentation (conservative)
augmentations:
  train:
    - name: resize
      height: 256
      width: 256
    - name: horizontal_flip
      p: 0.5
    - name: normalize
  val:
    - name: resize
      height: 256
      width: 256
    - name: normalize
"""
print("=== Light Augmentation ===")
print(aug_light)

In [ ]:
aug_medium = """
# Medium augmentation
augmentations:
  train:
    - name: resize
      height: 256
      width: 256
    - name: horizontal_flip
      p: 0.5
    - name: vertical_flip
      p: 0.5
    - name: rotate
      limit: 30
      p: 0.5
    - name: brightness_contrast
      brightness_limit: 0.2
      contrast_limit: 0.2
      p: 0.5
    - name: normalize
  val:
    - name: resize
      height: 256
      width: 256
    - name: normalize
"""
print("=== Medium Augmentation ===")
print(aug_medium)

In [ ]:
aug_heavy = """
# Heavy augmentation (aggressive)
augmentations:
  train:
    - name: resize
      height: 256
      width: 256
    - name: horizontal_flip
      p: 0.5
    - name: vertical_flip
      p: 0.5
    - name: rotate
      limit: 45
      p: 0.7
    - name: shift_scale_rotate
      shift_limit: 0.1
      scale_limit: 0.2
      rotate_limit: 0
      p: 0.5
    - name: brightness_contrast
      brightness_limit: 0.3
      contrast_limit: 0.3
      p: 0.5
    - name: hue_saturation
      hue_shift_limit: 20
      sat_shift_limit: 30
      val_shift_limit: 20
      p: 0.5
    - name: blur
      blur_limit: 3
      p: 0.3
    - name: gaussian_noise
      var_limit: [10, 50]
      p: 0.3
    - name: normalize
  val:
    - name: resize
      height: 256
      width: 256
    - name: normalize
"""
print("=== Heavy Augmentation ===")
print(aug_heavy)

## Task Types

### Binary Segmentation

In [ ]:
binary_config = """
# Binary segmentation (foreground vs background)
model:
  architecture: "unet"
  encoder: "resnet34"
  num_classes: 1        # Single output channel
  task: "binary"        # Sigmoid activation

training:
  loss: "dice"          # Good for binary

evaluation:
  thresh: 0.5           # Threshold for prediction

checkpointing:
  monitor: "val/Dice"   # Binary metrics
  mode: "max"
"""
print(binary_config)

### Multi-class Segmentation

In [ ]:
multiclass_config = """
# Multi-class segmentation (multiple classes)
model:
  architecture: "unet"
  encoder: "resnet50"
  num_classes: 10       # Number of classes
  task: "multiclass"    # Softmax activation

training:
  loss: "ce"            # Cross-entropy
  # Or combined
  # loss: "combined"
  # loss_weights:
  #   ce: 1.0
  #   dice: 0.5

checkpointing:
  monitor: "val/mIoU"   # Multi-class metrics
  mode: "max"
"""
print(multiclass_config)

### Regression

In [ ]:
regression_config = """
# Regression (continuous output)
# Example: depth estimation, density maps
model:
  architecture: "unet"
  encoder: "resnet34"
  num_classes: 1        # Single output channel
  task: "regression"    # No activation (linear output)

training:
  loss: "mse"           # Mean squared error
  # Or
  # loss: "l1"          # Mean absolute error

checkpointing:
  monitor: "val/MSE"    # Regression metrics
  mode: "min"           # Lower is better
"""
print(regression_config)

## Data Formats

### PNG Format

In [ ]:
png_config = """
# PNG mask format
# - Images and masks in separate directories
# - Matching filenames (image.png -> image.png)
# - Masks are grayscale with class indices as pixel values

data:
  format: "png"
  train_images: "data/train/images"
  train_masks: "data/train/masks"
  val_images: "data/val/images"
  val_masks: "data/val/masks"

# Directory structure:
# data/
#   train/
#     images/
#       001.png
#       002.png
#     masks/
#       001.png  (grayscale, values 0-N for N classes)
#       002.png
#   val/
#     ...
"""
print(png_config)

### COCO Format

In [ ]:
coco_config = """
# COCO format
# - Annotations in JSON file
# - Supports polygons and RLE masks

data:
  format: "coco"
  train_images: "data/train/images"
  train_annotations: "data/train/annotations.json"
  val_images: "data/val/images"
  val_annotations: "data/val/annotations.json"

# COCO JSON structure:
# {
#   "images": [{"id": 1, "file_name": "001.png", ...}],
#   "annotations": [{"id": 1, "image_id": 1, "category_id": 1, "segmentation": [...]}],
#   "categories": [{"id": 1, "name": "class1"}, ...]
# }
"""
print(coco_config)

## Real-World Examples

### Medical Image Segmentation

In [ ]:
medical_config = """
experiment:
  name: "tumor_segmentation"
  project: "medical_imaging"

model:
  architecture: "unet"
  encoder: "efficientnet_b3"
  encoder_weights: "imagenet"
  num_classes: 1
  task: "binary"
  decoder_attention: "scse"

data:
  format: "png"
  train_images: "data/train/images"
  train_masks: "data/train/masks"
  val_images: "data/val/images"
  val_masks: "data/val/masks"
  batch_size: 16

training:
  epochs: 200
  lr: 0.0001
  optimizer: "adamw"
  scheduler: "cosine"
  loss: "combined"
  loss_weights:
    dice: 1.0
    bce: 0.5

augmentations:
  train:
    - name: resize
      height: 512
      width: 512
    - name: horizontal_flip
      p: 0.5
    - name: vertical_flip
      p: 0.5
    - name: rotate
      limit: 15
      p: 0.5
    - name: elastic_transform
      alpha: 120
      sigma: 6
      p: 0.3
    - name: normalize
  val:
    - name: resize
      height: 512
      width: 512
    - name: normalize
"""
print(medical_config)

### Satellite Image Segmentation

In [ ]:
satellite_config = """
experiment:
  name: "land_cover"
  project: "remote_sensing"

model:
  architecture: "unetplusplus"
  encoder: "resnet50"
  encoder_weights: "imagenet"
  num_classes: 7  # water, forest, urban, etc.
  task: "multiclass"

data:
  format: "png"
  train_images: "data/train/images"
  train_masks: "data/train/masks"
  val_images: "data/val/images"
  val_masks: "data/val/masks"
  batch_size: 8

training:
  epochs: 150
  lr: 0.001
  optimizer: "adamw"
  scheduler: "cosine"
  loss: "combined"
  loss_weights:
    ce: 1.0
    dice: 0.5

augmentations:
  train:
    - name: resize
      height: 256
      width: 256
    - name: horizontal_flip
      p: 0.5
    - name: vertical_flip
      p: 0.5
    - name: rotate
      limit: 90
      p: 0.5
    - name: random_brightness_contrast
      p: 0.5
    - name: normalize
  val:
    - name: resize
      height: 256
      width: 256
    - name: normalize

checkpointing:
  monitor: "val/mIoU"
  mode: "max"
"""
print(satellite_config)

## Programmatic Configuration

You can also build configs programmatically:

In [ ]:
programmatic_example = """
from altair.core.config import Config

# Build config as dictionary
config = {
    "experiment": {
        "name": "test_experiment",
        "project": "testing",
    },
    "model": {
        "architecture": "unet",
        "encoder": "resnet34",
        "num_classes": 2,
        "task": "binary",
    },
    # ... rest of config
}

# Validate config
validated_config = Config.from_dict(config)

# Access config values
print(validated_config.model.encoder)
print(validated_config.training.lr)

# Convert back to dict
config_dict = validated_config.to_dict()

# Save to YAML
validated_config.to_yaml("my_config.yaml")
"""
print(programmatic_example)

## Summary

Key configuration tips:

1. **Start simple**: Begin with default settings and iterate
2. **Use pretrained encoders**: Almost always improves results
3. **Match loss to task**: Dice for imbalanced, CE for balanced
4. **Augment appropriately**: Don't over-augment for small datasets
5. **Monitor the right metric**: Dice/IoU for segmentation quality
6. **Use mixed precision**: Enable `amp: true` for faster training